In [1]:
import sqlite3
# import ibis
import sys
sys.path.append("..")
from rcn_py import database
from rcn_py import data_storage

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



    author-paper table
    author info table
    publication table

In [10]:
con = sqlite3.connect("tutorial.db")
cur = con.cursor()

In [12]:
cur.execute("CREATE TABLE test_paper(doi primary key, title, abstract)")

In [13]:
cur.execute("CREATE TABLE test_authors(orcid primary key, name, country)")


In [16]:
cur.execute("CREATE TABLE test_author_papers2(orcid, doi, primary key(orcid, doi) FOREIGN KEY(orcid) REFERENCES test_author(orcid), FOREIGN KEY(doi) REFERENCES test_paper(doi))")

In [55]:
con.commit()

In [56]:
con.close()

## Insert data

In [72]:
database.insert_database("/Users/jennifer/RCSNA/data/tutorial.db", '',"Peter Kalverla")

'Done.'

In [ ]:
database.insert_coauthors_pub("/Users/jennifer/RCSNA/data/tutorial.db", "Peter Kalverla")

In [ ]:
database.insert_cocoauthors("/Users/jennifer/RCSNA/data/tutorial.db")

### Fetch data from database

In [ ]:
con = sqlite3.connect("/Users/jennifer/RCSNA/data/tutorial.db")
cur = con.cursor()

# Topic modeling (title or abstract) to cluster papers
clusters, idx2topics = database.pub_cluster(cur)

In [ ]:
# get coauthor relationship
links = database.fetch_relationships(cur)

In [ ]:
# assign author's group as the most frequent paper group
orcid_group = database.author_cluster(cur, clusters)

cur.close()

In [ ]:
# pyvis
database.build_network_database("Database")

### Store more data

In [4]:
con = sqlite3.connect("/Users/jennifer/RCSNA/data/tutorial.db")
cur = con.cursor()

In [6]:
con_rsd = sqlite3.connect("/Users/jennifer/RCSNA/data/rsd.db")
cur_rsd = con_rsd.cursor()

In [18]:
res_rsd = cur_rsd.execute("""
SELECT orcid, given_name, family_name, affiliation 
FROM author 
WHERE affiliation = 'Netherlands eScience Center'
GROUP BY orcid
UNION
SELECT orcid, given_name, family_name, affiliation 
FROM contributor 
WHERE affiliation = 'Netherlands eScience Center'
GROUP BY orcid
""")
escience_member = res_rsd.fetchall()

In [25]:
escience_member[0]


('0000-0001-5301-3276',
 'Jesus',
 'Garcia Gonzalez',
 'Netherlands eScience Center')

In [26]:
def insert_escience_rsd_member(db_path):
    con_rsd = sqlite3.connect("/Users/jennifer/RCSNA/data/rsd.db")
    cur_rsd = con_rsd.cursor()
    res_rsd = cur_rsd.execute("""
        SELECT orcid, given_name, family_name, affiliation 
        FROM author 
        WHERE affiliation = 'Netherlands eScience Center'
        GROUP BY orcid
        UNION
        SELECT orcid, given_name, family_name, affiliation 
        FROM contributor 
        WHERE affiliation = 'Netherlands eScience Center'
        GROUP BY orcid
        """)
    escience_members = res_rsd.fetchall()
    for member in escience_members:
        orcid_id = member[0]
        fullname = member[1] + " " + member[2]
        database.insert_database(db_path, orcid_id, fullname)

    cur_rsd.commit()
    cur_rsd.close()
    return "Insertion complete"
    

In [ ]:
insert_escience_rsd_member()